Some work were followed by the advice of https://nik-davis.github.io/posts/2019/steam-data-cleaning/

... and based on the information on the dataset some decisions are made

In [223]:
# loaded required libraries

import pandas as pd
import numpy as np
import json
from ast import literal_eval
import re

## Import and clean the steam store data

In [224]:
steam_basic = pd.read_json("steam_11_2022/steam_games.json", orient = 'index')

In [225]:
steam_basic = steam_basic.reset_index(drop=True)

In [226]:
steam_basic.head(3)

,appid,name,short_description,developer,publisher,genre,tags,type,categories,owners,...,price,initialprice,discount,ccu,languages,platforms,release_date,required_age,website,header_image
0,10,Counter-Strike,Play the world's number 1 online action game. ...,Valve,Valve,Action,"{'Action': 5426, 'FPS': 4831, 'Multiplayer': 3...",game,"[Multi-player, Valve Anti-Cheat enabled, Onlin...","10,000,000 .. 20,000,000",...,999,999,0,13990,"English, French, German, Italian, Spanish - Sp...","{'windows': True, 'mac': True, 'linux': True}",2000/11/1,0,,https://cdn.akamai.steamstatic.com/steam/apps/...
1,1000000,ASCENXION,ASCENXION is a 2D shoot 'em up game where you ...,IndigoBlue Game Studio,PsychoFlux Entertainment,"Action, Adventure, Indie","{'Shoot 'Em Up': 186, 'Metroidvania': 181, 'Bu...",game,"[Single-player, Partial Controller Support, St...","0 .. 20,000",...,999,999,0,0,"English, Korean, Simplified Chinese","{'windows': True, 'mac': False, 'linux': False}",2021/05/14,0,,https://cdn.akamai.steamstatic.com/steam/apps/...
2,1000010,Crown Trick,"Enter a labyrinth that moves as you move, wher...",NEXT Studios,"Team17, NEXT Studios","Adventure, Indie, RPG, Strategy","{'Rogue-like': 268, 'Turn-Based Combat': 254, ...",game,"[Single-player, Partial Controller Support, St...","200,000 .. 500,000",...,599,1999,70,99,"Simplified Chinese, English, Japanese, Traditi...","{'windows': True, 'mac': False, 'linux': False}",2020/10/16,0,,https://cdn.akamai.steamstatic.com/steam/apps/...


## Combine the two data

Now we have 2 datasets... - we need to combine them

the reason why we didnt combine them in the developer and requirement is...

In [227]:
steam_basic.shape

(55691, 22)

In [228]:
steam_csv = pd.read_csv('steam_dlc/steam.csv')

In [229]:
steam_csv.shape

(102504, 26)

The *steam_basic* contains games only on the Steam Store website. The *steam_csv* data table contains additional dlc information. Although the first one only contain games on the steam store, it is the most updated version of the all the games available on the store by November 2022. Since we also need dlc information, that is why we acquired another dataset which is the second one.

DLC means Downloadable Content. They are not games but additional contents downloadable for the "parent" game. Also, since the second datasets contain information far more than the first dataset, we would like to combine those two sets of data. Our ER diagram is built **upon 2 main data tables**, one is **steam game info data table**, another is **dlc info table**. They both contains similar basic information: name, release_date, etc. However, dlc table would contain additional information of its "parent game name" and "parent game id". **Parent game id serves as the foreign key to the steam game info data table**. 

**What we are doing in the following:**
1. Keep the whole steam_basic table, extract appid, name, release_date, tag_lists, and developer to be the steam_game pandas dataframe. Clean this data.

2. Split the DLC out of steam_csv to be DLC table, containing attribures of name, release_date, tag_lists, and developer to be the dlc pandas dataframe. Clean this data. 

3. On dlc, add attributes parent_id, parent_name. Parent_id serves as the foreign key referring to the appid of the steam_game table. This means that the referred game in the steam_game table is the parent game for each dlc. 

4. Clean the two datasets to be in the same data format.

In [230]:
steam_basic.head(3)

,appid,name,short_description,developer,publisher,genre,tags,type,categories,owners,...,price,initialprice,discount,ccu,languages,platforms,release_date,required_age,website,header_image
0,10,Counter-Strike,Play the world's number 1 online action game. ...,Valve,Valve,Action,"{'Action': 5426, 'FPS': 4831, 'Multiplayer': 3...",game,"[Multi-player, Valve Anti-Cheat enabled, Onlin...","10,000,000 .. 20,000,000",...,999,999,0,13990,"English, French, German, Italian, Spanish - Sp...","{'windows': True, 'mac': True, 'linux': True}",2000/11/1,0,,https://cdn.akamai.steamstatic.com/steam/apps/...
1,1000000,ASCENXION,ASCENXION is a 2D shoot 'em up game where you ...,IndigoBlue Game Studio,PsychoFlux Entertainment,"Action, Adventure, Indie","{'Shoot 'Em Up': 186, 'Metroidvania': 181, 'Bu...",game,"[Single-player, Partial Controller Support, St...","0 .. 20,000",...,999,999,0,0,"English, Korean, Simplified Chinese","{'windows': True, 'mac': False, 'linux': False}",2021/05/14,0,,https://cdn.akamai.steamstatic.com/steam/apps/...
2,1000010,Crown Trick,"Enter a labyrinth that moves as you move, wher...",NEXT Studios,"Team17, NEXT Studios","Adventure, Indie, RPG, Strategy","{'Rogue-like': 268, 'Turn-Based Combat': 254, ...",game,"[Single-player, Partial Controller Support, St...","200,000 .. 500,000",...,599,1999,70,99,"Simplified Chinese, English, Japanese, Traditi...","{'windows': True, 'mac': False, 'linux': False}",2020/10/16,0,,https://cdn.akamai.steamstatic.com/steam/apps/...


In [231]:
# Inpect if there are null values
steam_basic.isnull().sum()

appid                0
name                 0
short_description    0
developer            0
publisher            0
genre                0
tags                 0
type                 0
categories           0
owners               0
positive             0
negative             0
price                0
initialprice         0
discount             0
ccu                  0
languages            0
platforms            0
release_date         0
required_age         0
website              0
header_image         0
dtype: int64

In [232]:
# Inspect if there are duplicated app ids
steam_basic.loc[:, 'appid'].duplicated().sum()

0

### Extract required columns of steam_basic and clean to prepare the  dataset to be ready as the stem game table in the ER diagram

In [233]:
steam_game = steam_basic.loc[:, ('appid', 'name', 'release_date', 
                                'genre', 'categories', 'tags', 'developer')]

In [234]:
steam_game.head(3)

,appid,name,release_date,genre,categories,tags,developer
0,10,Counter-Strike,2000/11/1,Action,"[Multi-player, Valve Anti-Cheat enabled, Onlin...","{'Action': 5426, 'FPS': 4831, 'Multiplayer': 3...",Valve
1,1000000,ASCENXION,2021/05/14,"Action, Adventure, Indie","[Single-player, Partial Controller Support, St...","{'Shoot 'Em Up': 186, 'Metroidvania': 181, 'Bu...",IndigoBlue Game Studio
2,1000010,Crown Trick,2020/10/16,"Adventure, Indie, RPG, Strategy","[Single-player, Partial Controller Support, St...","{'Rogue-like': 268, 'Turn-Based Combat': 254, ...",NEXT Studios


We would like to check the data type of the release_date and tags.

In [235]:
print('release_date:', type(steam_game['release_date'][0]))
print('tags:', type(steam_game['tags'][0]))
print('categories:', type(steam_game['categories'][0]))
print('genre:', type(steam_game['genre'][0]))

release_date: <class 'str'>
tags: <class 'dict'>
categories: <class 'list'>
genre: <class 'str'>


**Clean the attribute 'tags'**


We would like to clean tag to be a string containing all the tags of this game, dismissing the votes.

In [236]:
tags = steam_game['tags']

In [237]:
tags.head()

0    {'Action': 5426, 'FPS': 4831, 'Multiplayer': 3...
1    {'Shoot 'Em Up': 186, 'Metroidvania': 181, 'Bu...
2    {'Rogue-like': 268, 'Turn-Based Combat': 254, ...
3    {'Typing': 221, 'Management': 213, 'Casual': 2...
4    {'Action': 22, 'Casual': 22, 'Indie': 21, 'Sim...
Name: tags, dtype: object

In [238]:
# to inspect if there is empty dictionaries
(steam_basic['tags'] == {}).sum()

135

In [240]:
tag_content = []
for n in tags:
    this_tag = ''
    for key in n.keys():
        this_tag += key + ';'
    tag_content.append(this_tag.strip()[0:-1])

Since there were 135 empty dictionaries, after cleaning like the aboved, those empty dictionaries would turn into '', which is the empty string. Therefore, we want to make those empty string into 'None'. 

In [242]:
for i in range(len(tag_content)):
    if tag_content[i] == '':
        tag_content[i] = 'None'

In [243]:
# append this to the steam_game dataset and drop the tag column
steam_game = steam_game.drop('tags', axis = 1)

In [244]:
steam_game.columns

Index(['appid', 'name', 'release_date', 'genre', 'categories', 'developer'], dtype='object')

In [245]:
steam_game['tags'] = tag_content

In [246]:
steam_game.head(3)

,appid,name,release_date,genre,categories,developer,tags
0,10,Counter-Strike,2000/11/1,Action,"[Multi-player, Valve Anti-Cheat enabled, Onlin...",Valve,Action;FPS;Multiplayer;Shooter;Classic;Team-Ba...
1,1000000,ASCENXION,2021/05/14,"Action, Adventure, Indie","[Single-player, Partial Controller Support, St...",IndigoBlue Game Studio,Shoot 'Em Up;Metroidvania;Bullet Hell;Side Scr...
2,1000010,Crown Trick,2020/10/16,"Adventure, Indie, RPG, Strategy","[Single-player, Partial Controller Support, St...",NEXT Studios,Rogue-like;Turn-Based Combat;RPG;Perma Death;R...


**Clean the attribute 'categories'**

We want it to be the same format as tags. It originally contains list object in python. We woule like to extract those strings out of the list of each row to form a new string of categories separated by ';'. If the list is empty, we put 'None' in this row. 

In [247]:
categories = steam_game['categories']

In [248]:
categories.tail()

55686                                                   []
55687                                      [Single-player]
55688                                                   []
55689    [Single-player, Partial Controller Support, St...
55690     [Single-player, Steam Achievements, Steam Cloud]
Name: categories, dtype: object

In [250]:
# define a function to unlist the categories into a string
def unlist(n):
    # n is a list, output a string of content in the list
    # separated by ';''
    string = ''
    for item in n:
        string += item + ';'
    return string[0:-1]

In [264]:
cleaned_categories = []
for item in categories:
    if item == []:
        cleaned_categories.append('None')
    else:
        cleaned_categories.append(unlist(item))

In [253]:
steam_game = steam_game.drop('categories', axis = 1)

In [254]:
steam_game['categories'] = cleaned_categories

In [255]:
steam_game.head(3)

,appid,name,release_date,genre,developer,tags,categories
0,10,Counter-Strike,2000/11/1,Action,Valve,Action;FPS;Multiplayer;Shooter;Classic;Team-Ba...,Multi-player;Valve Anti-Cheat enabled;Online P...
1,1000000,ASCENXION,2021/05/14,"Action, Adventure, Indie",IndigoBlue Game Studio,Shoot 'Em Up;Metroidvania;Bullet Hell;Side Scr...,Single-player;Partial Controller Support;Steam...
2,1000010,Crown Trick,2020/10/16,"Adventure, Indie, RPG, Strategy",NEXT Studios,Rogue-like;Turn-Based Combat;RPG;Perma Death;R...,Single-player;Partial Controller Support;Steam...


We don't need clean genre since it has been cleaned in the original dataset to be into strings without null values. However, we do need reformat this column to switch the ',' to ';' so as to be consistent with othe columns

In [256]:
steam_game['genre'][:4]

0                                 Action
1               Action, Adventure, Indie
2        Adventure, Indie, RPG, Strategy
3    Action, Indie, Simulation, Strategy
Name: genre, dtype: object

In [259]:
steam_game['genre'][2].replace(',', ';').replace(' ', '')

'Adventure;Indie;RPG;Strategy'

In [261]:
(steam_game['genre'] == '').sum()

161

In [267]:
cleaned_genre = []
for item in steam_game['genre']:
    if item == '':
        cleaned_genre.append('None')
    else:
        cleaned_genre.append(item.replace(',', ';').replace(' ', ''))

In [268]:
cleaned_genre[:4]

['Action',
 'Action;Adventure;Indie',
 'Adventure;Indie;RPG;Strategy',
 'Action;Indie;Simulation;Strategy']

In [269]:
steam_game.drop('genre', axis = 1)
steam_game['genre'] = cleaned_genre

In [278]:
steam_game.head(3)

,appid,name,release_date,genre,developer,tags,categories
0,10,Counter-Strike,2000/11/1,Action,Valve,Action;FPS;Multiplayer;Shooter;Classic;Team-Ba...,Multi-player;Valve Anti-Cheat enabled;Online P...
1,1000000,ASCENXION,2021/05/14,Action;Adventure;Indie,IndigoBlue Game Studio,Shoot 'Em Up;Metroidvania;Bullet Hell;Side Scr...,Single-player;Partial Controller Support;Steam...
2,1000010,Crown Trick,2020/10/16,Adventure;Indie;RPG;Strategy,NEXT Studios,Rogue-like;Turn-Based Combat;RPG;Perma Death;R...,Single-player;Partial Controller Support;Steam...


**Now steam game table should be ready. However, according to the ER diagram, since we still need to make the developer table, now we just leave this dataset as it is and modify the developer column later when making the Developer table.**

### Split the dlc data from steam_csv and clean to prepare the dataset to be ready as the dlc table in the ER diagram

In [279]:
steam_csv.head(3)

,appid,type,name,required_age,dlc,fullgame,supported_languages,developers,publishers,packages,...,coming_soon,price,review_score,total_positive,total_negative,rating,owners,average_forever,median_forever,tags
0,10,game,Counter-Strike,0,NaN,NaN,"['English', 'French', 'German', 'Italian', 'Ko...",['Valve'],['Valve'],"[574941, 7]",...,False,8.19,9.0,117261.0,3686.0,95.566768,10000000-20000000,10499,202,"['Action', 'FPS', 'Multiplayer', 'Shooter', 'C..."
1,20,game,Team Fortress Classic,0,NaN,NaN,"['English', 'French', 'German', 'Italian', 'Ko...",['Valve'],['Valve'],[29],...,False,3.99,8.0,3896.0,705.0,81.939532,2000000-5000000,1637,23,"['Action', 'FPS', 'Multiplayer', 'Classic', 'H..."
2,30,game,Day of Defeat,0,NaN,NaN,"['English', 'French', 'German', 'Italian', 'Sp...",['Valve'],['Valve'],[30],...,False,3.99,8.0,2794.0,398.0,84.223637,5000000-10000000,169,11,"['FPS', 'World War II', 'Multiplayer', 'Shoote..."


In [280]:
steam_csv.isnull().sum()

appid                      0
type                       0
name                       0
required_age               0
dlc                    92808
fullgame               67897
supported_languages      152
developers                41
publishers                40
packages               21351
platforms                  0
categories               106
genres                   193
achievements               0
release_date            6828
supported_audio        52806
coming_soon                0
price                  11834
review_score               0
total_positive             0
total_negative             0
rating                     0
owners                     0
average_forever            0
median_forever             0
tags                   41456
dtype: int64

In [281]:
steam_csv['type'].value_counts()

game           67870
dlc            34632
advertising        1
music              1
Name: type, dtype: int64

Accroding to the description of this dataset, the column *type* contains information of whether this is a "game" or just "dlc" of its parent game. Therefore, we extract rows that are dlc. 'music' and 'advertising' are dismissed. 

In [282]:
dlc = steam_csv.query('type == "dlc"')

In [283]:
dlc.columns

Index(['appid', 'type', 'name', 'required_age', 'dlc', 'fullgame',
       'supported_languages', 'developers', 'publishers', 'packages',
       'platforms', 'categories', 'genres', 'achievements', 'release_date',
       'supported_audio', 'coming_soon', 'price', 'review_score',
       'total_positive', 'total_negative', 'rating', 'owners',
       'average_forever', 'median_forever', 'tags'],
      dtype='object')

In [284]:
dlc.shape

(34632, 26)

In [285]:
dlc = dlc.loc[:, ('appid', 'name', 'release_date', 'genres', 'categories', 'tags', 'developers')]

In [286]:
dlc.shape

(34632, 7)

In [287]:
dlc.head(3)

,appid,name,release_date,genres,categories,tags,developers
29,1256,Killing Floor Outbreak Character Pack,2009-07-24,['Action'],"['Single-player', 'Multi-player', 'Co-op', 'Do...",['Action'],['Tripwire Interactive']
30,1257,Killing Floor: Nightfall Character Pack,2010-05-19,['Action'],"['Single-player', 'Multi-player', 'Co-op', 'Do...",['Action'],['Tripwire Interactive']
162,4856,Cossacks: Campaign Expansion,2011-01-21,['Strategy'],"['Single-player', 'Downloadable Content']",['Strategy'],['GSC Game World']


In [288]:
# reset the index
dlc = dlc.reset_index(drop = True)

In [289]:
dlc.head(3)

,appid,name,release_date,genres,categories,tags,developers
0,1256,Killing Floor Outbreak Character Pack,2009-07-24,['Action'],"['Single-player', 'Multi-player', 'Co-op', 'Do...",['Action'],['Tripwire Interactive']
1,1257,Killing Floor: Nightfall Character Pack,2010-05-19,['Action'],"['Single-player', 'Multi-player', 'Co-op', 'Do...",['Action'],['Tripwire Interactive']
2,4856,Cossacks: Campaign Expansion,2011-01-21,['Strategy'],"['Single-player', 'Downloadable Content']",['Strategy'],['GSC Game World']


In [290]:
dlc.isnull().sum()

appid               0
name                0
release_date      323
genres             89
categories          0
tags            24188
developers         13
dtype: int64

In [291]:
dlc.loc[:, 'appid'].duplicated().sum()

0

We see that we need to further investigate the null values of dlc and see what happen. Also, we need re-format the *tags*, *categories*, *genres*, and *developers* columns to be consistent with the steam game table. 

In [292]:
# change nan in release_date to not available
dlc.loc[dlc['release_date'].isnull(), 'release_date'] = 'Not Available'

In [293]:
dlc.isnull().sum()

appid               0
name                0
release_date        0
genres             89
categories          0
tags            24188
developers         13
dtype: int64

In [294]:
print('release_date:', type(dlc['release_date'][0]))
print('tags:', type(dlc['tags'][0]))
print('categories:', type(dlc['categories'][0]))
print('genre:', type(dlc['genres'][0]))
print('developers:', type(dlc['developers'][0]))

release_date: <class 'str'>
tags: <class 'str'>
categories: <class 'str'>
genre: <class 'str'>
developers: <class 'str'>


In [295]:
dlc.loc[dlc['developers'].isnull(), :].head(1)

,appid,name,release_date,genres,categories,tags,developers
244,50142,Mafia II - Vegas DLC,Not Available,NaN,['Downloadable Content'],NaN,NaN


Change all NAN in the attributes *genres*, *tags*, and *developers* to be 'None'

In [296]:
# change nan in tags,  to be 'None'
dlc.loc[dlc['tags'].isnull(), 'tags'] = 'None'
dlc.loc[dlc['genres'].isnull(), 'genres'] = 'None'
dlc.loc[dlc['developers'].isnull(), 'developers'] = 'None'

In [297]:
dlc.isnull().sum()

appid           0
name            0
release_date    0
genres          0
categories      0
tags            0
developers      0
dtype: int64

Reformat genres, categories, developers, and tags to be the same format respectively as the steam_game dataset. 

In [298]:
dlc.head(1)

,appid,name,release_date,genres,categories,tags,developers
0,1256,Killing Floor Outbreak Character Pack,2009-07-24,['Action'],"['Single-player', 'Multi-player', 'Co-op', 'Do...",['Action'],['Tripwire Interactive']


In [352]:
dlc['genres'].head(7)

0           ['Action']
1           ['Action']
2         ['Strategy']
3           ['Racing']
4           ['Racing']
5           ['Racing']
6    ['Action', 'RPG']
Name: genres, dtype: object

In [353]:
dlc_cleaned_genres = []

for item in dlc['genres']:
    if item != 'None':
        item = literal_eval(item)
        dlc_cleaned_genres.append(unlist(item))
    else:
        dlc_cleaned_genres.append(item)

In [354]:
dlc_cleaned_genres[:7]

['Action', 'Action', 'Strategy', 'Racing', 'Racing', 'Racing', 'Action;RPG']

In [355]:
dlc_cleaned_categories = []

for item in dlc['categories']:
    if item != 'None':
        item = literal_eval(item)
        dlc_cleaned_categories.append(unlist(item))
    else:
        dlc_cleaned_categories.append(item)

In [356]:
dlc_cleaned_categories[:6]

['Single-player;Multi-player;Co-op;Downloadable Content',
 'Single-player;Multi-player;Co-op;Downloadable Content',
 'Single-player;Downloadable Content',
 'Downloadable Content',
 'Single-player;Multi-player;Downloadable Content',
 'Single-player;Multi-player;Downloadable Content']

In [358]:
dlc_cleaned_tags = []
for item in dlc['tags']:
    if item != 'None':
        item = literal_eval(item)
        dlc_cleaned_tags.append(unlist(item))
    else:
        dlc_cleaned_tags.append(item)

In [359]:
dlc_cleaned_tags[:7]

['Action',
 'Action',
 'Strategy',
 'Racing;Simulation;Shooter',
 'Racing;Simulation;Driving;Multiplayer;Singleplayer',
 'Racing;Simulation',
 'RPG;Action;Zombies']

In [360]:
dlc_new = dlc.drop(['genres', 'categories', 'tags'], axis = 1)

In [361]:
dlc_new.columns

Index(['appid', 'name', 'release_date', 'developers'], dtype='object')

In [363]:
dlc_new['genre'] = dlc_cleaned_genres
dlc_new['categories'] = dlc_cleaned_categories
dlc_new['tags'] = dlc_cleaned_tags

In [365]:
dlc_new.head(7)

,appid,name,release_date,developers,genre,categories,tags
0,1256,Killing Floor Outbreak Character Pack,2009-07-24,['Tripwire Interactive'],Action,Single-player;Multi-player;Co-op;Downloadable ...,Action
1,1257,Killing Floor: Nightfall Character Pack,2010-05-19,['Tripwire Interactive'],Action,Single-player;Multi-player;Co-op;Downloadable ...,Action
2,4856,Cossacks: Campaign Expansion,2011-01-21,['GSC Game World'],Strategy,Single-player;Downloadable Content,Strategy
3,8650,RACE 07: Andy Priaulx Crowne Plaza Raceway (Fr...,2008-06-25,['SimBin'],Racing,Downloadable Content,Racing;Simulation;Shooter
4,8660,GTR Evolution Expansion Pack for RACE 07,2008-09-01,['SimBin'],Racing,Single-player;Multi-player;Downloadable Content,Racing;Simulation;Driving;Multiplayer;Singlepl...
5,8690,STCC - The Game 1 - Expansion Pack for RACE 07,2008-10-22,['SimBin'],Racing,Single-player;Multi-player;Downloadable Content,Racing;Simulation
6,8990,Borderlands: The Zombie Island of Dr. Ned,2009-12-08,['Gearbox Software'],Action;RPG,Single-player;Multi-player;Co-op;Downloadable ...,RPG;Action;Zombies


**Now it is ready for dlc table. Since we will make Developer table later and will combine the 'developers' column in dlc_new with the 'developers' column in steam_game, we will export this dataset later after the modification is done.**

## Developer Table

What we are going to do in the followings:

1. extract the appid and developers columns in steam_game and dlc_new as two new datasets, respectively called steam_developers and dlc_developers.
2. clean the developers column of both two datasets to be in the same format. Developer name should be in string. If there are more than one developers, their name should be separated with ';' in a string. We named the cleaned datasets as new_steam_developers and new_dlc_developers.
3. keep new_steam_developers and merge the new_dlc_developers data into the new_steam_developers. Drop the rows of duplicated appid. Rename this new dataset as developer.

### Extract

In [367]:
steam_developers = steam_game[['appid', 'developer']]

In [368]:
steam_developers.head(3)

,appid,developer
0,10,Valve
1,1000000,IndigoBlue Game Studio
2,1000010,NEXT Studios


In [370]:
dlc_developers = dlc_new[['appid', 'developers']]

In [371]:
dlc_developers.head(3)

,appid,developers
0,1256,['Tripwire Interactive']
1,1257,['Tripwire Interactive']
2,4856,['GSC Game World']


### Clean

In [375]:
print('developers type in steam_developers:', type(steam_developers['developer'][0]))
print('developers type in dlc_new:', type(dlc_new['developers'][0]))

developers type in steam_developers: <class 'str'>
developers type in dlc_new: <class 'str'>


**clean steam_developer**

In [377]:
steam_developers['developer']

0                                                 Valve
1                                IndigoBlue Game Studio
2                                          NEXT Studios
3                                   Vertigo Gaming Inc.
4                                         DoubleC Games
                              ...                      
55686    Caltech/IPAC, NASA's Jet Propulsion Laboratory
55687                                    FlairBot Games
55688                                           RADiCAL
55689                                            张八万工作室
55690                                    ZPink, zniq.co
Name: developer, Length: 55691, dtype: object

In [384]:
print('rows contain no developer name (\'\'):', steam_developers.loc[steam_developers['developer'] == '', :].shape[0])

rows contain no developer name (''): 127


In [385]:
steam_cleaned_developers = []
for item in steam_developers['developer']:
    if item == '':
        steam_cleaned_developers.append('Not Available')
    else:
        steam_cleaned_developers.append(item.replace(', ', ';'))

In [389]:
steam_cleaned_developers[55690]

'ZPink;zniq.co'

In [390]:
new_steam_developers = steam_developers.drop('developer', axis = 1)

In [391]:
new_steam_developers['developers'] = steam_cleaned_developers

In [392]:
new_steam_developers.head(3)

,appid,developers
0,10,Valve
1,1000000,IndigoBlue Game Studio
2,1000010,NEXT Studios


In [455]:
print('rows contain no developer name:', dlc_developers.loc[dlc_developers['developers'] == '[\'\']', ].shape[0])

rows contain no developer name: 0


In [450]:
dlc_developers.loc[dlc_developers['developers'] == '[\'\']', ].shape[0]

0

**clean dlc_developers**

In [420]:
dlc_developers['developers'][34620:34629]

34620                       ['Cyberfunk Studios LLC']
34621                              ['Positech Games']
34622                          ['Falcon Development']
34623                          ['Falcon Development']
34624                                  ['bananadev.']
34625                                ['Corey Martin']
34626                                   ['King Moon']
34627                       ['The Creative Assembly']
34628    ['Ubisoft Paris', 'Red Storm Entertainment']
Name: developers, dtype: object

In [427]:
dlc_developers['developers'][34620:34629].str.replace(r'[\[\]\'\']', '').str.replace(', ', ';')

/var/folders/7g/xwbj7h6n29q8t527ck_zttsw0000gn/T/ipykernel_65420/2329402228.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  dlc_developers['developers'][34620:34629].str.replace(r'[\[\]\'\']', '').str.replace(', ', ';')


34620                    Cyberfunk Studios LLC
34621                           Positech Games
34622                       Falcon Development
34623                       Falcon Development
34624                               bananadev.
34625                             Corey Martin
34626                                King Moon
34627                    The Creative Assembly
34628    Ubisoft Paris;Red Storm Entertainment
Name: developers, dtype: object

In [456]:
dlc_cleaned_developers = dlc_developers['developers'].str.replace(r'[\[\]\'\']', '').str.replace(', ', ';')

/var/folders/7g/xwbj7h6n29q8t527ck_zttsw0000gn/T/ipykernel_65420/3148450432.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  dlc_cleaned_developers = dlc_developers['developers'].str.replace(r'[\[\]\'\']', '').str.replace(', ', ';')


In [457]:
dlc_cleaned_developers.head(3)

0    Tripwire Interactive
1    Tripwire Interactive
2          GSC Game World
Name: developers, dtype: object

In [458]:
new_dlc_developers = dlc_developers.drop('developers', axis = 1)

In [460]:
new_dlc_developers['developers'] = dlc_cleaned_developers

In [461]:
new_dlc_developers.head(3)

,appid,developers
0,1256,Tripwire Interactive
1,1257,Tripwire Interactive
2,4856,GSC Game World


### Merge and Combine

In [464]:
new_dlc_developers['appid']

0           1256
1           1257
2           4856
3           8650
4           8660
          ...   
34627    2028023
34628    2028055
34629    2028056
34630    2028062
34631    2028850
Name: appid, Length: 34632, dtype: int64

In [465]:
new_steam_developers['appid']

0             10
1        1000000
2        1000010
3        1000030
4        1000040
          ...   
55686     999880
55687     999890
55688     999900
55689     999930
55690     999990
Name: appid, Length: 55691, dtype: int64

In [485]:
# before concatenation, check if there are overlapping appids
appids = pd.concat([new_steam_developers['appid'], new_dlc_developers['appid']], ignore_index = True, join = 'inner')
print('Overlaping appid:', appids.duplicated().sum())

Overlaping appid: 0


In [486]:
# Since there is no overlapping appid, we can directly concat the 
# two data frame
developers = pd.concat([new_steam_developers, new_dlc_developers], ignore_index = True, join = 'inner')

### Map developers with developer id and form the final developer table

In [511]:
dev_dict = {}
for item in developers['developers']:
    for n in item.split(';'):
        if n in dev_dict.keys():
            dev_dict[n] += 1
        else:
            dev_dict[n] = 1

In [513]:
developer_names = [n for n in dev_dict.keys()]

In [515]:
developer_id = [i + 1 for i in range(len(developer_names))]

In [520]:
Developer = pd.DataFrame({
    'developer_id': developer_id,
    'developer_name': developer_names
})

In [521]:
Developer.head()

,developer_id,developer_name
0,1,Valve
1,2,IndigoBlue Game Studio
2,3,NEXT Studios
3,4,Vertigo Gaming Inc.
4,5,DoubleC Games


This is the Developer table. However, before we export it, we need to change the 'developers' columns in the steam_game and dlc_new dataset from containing developer_name to tupple of mapped developer_id.

### Mapping id in the steam_gamd and developer_name datasets

In [544]:
tuppled_steam_developers = []
for item in steam_cleaned_developers:
    ids = []
    for n in item.split(';'):
        ids.append(int(Developer.loc[Developer['developer_name'] == n, 'developer_id']))
    tuppled_steam_developers.append(tuple(ids))

In [546]:
tuppled_dlc_developers = []
for item in dlc_cleaned_developers:
    ids = []
    for n in item.split(';'):
        ids.append(int(Developer.loc[Developer['developer_name'] == n, 'developer_id']))
    tuppled_dlc_developers.append(tuple(ids))

Up to this point, we have finishing cleaning Developer table. Now we switch the developers column in the steam_game and dlc_new table, and expeort them to be the finished steam_game.csv and dlc.csv.

Export Developer to be developer.csv

In [550]:
steam_game['developer_id'] = tuppled_steam_developers

In [551]:
dlc_new['developer_id'] = tuppled_dlc_developers

In [556]:
steam_game = steam_game.drop('developer', axis = 1)
dlc_new = dlc_new.drop('developers', axis = 1)

In [559]:
steam_game.head(2)

,appid,name,release_date,genre,tags,categories,developer_id
0,10,Counter-Strike,2000/11/1,Action,Action;FPS;Multiplayer;Shooter;Classic;Team-Ba...,Multi-player;Valve Anti-Cheat enabled;Online P...,"(1,)"
1,1000000,ASCENXION,2021/05/14,Action;Adventure;Indie,Shoot 'Em Up;Metroidvania;Bullet Hell;Side Scr...,Single-player;Partial Controller Support;Steam...,"(2,)"


In [560]:
dlc_new.head(2)

,appid,name,release_date,genre,categories,tags,developer_id
0,1256,Killing Floor Outbreak Character Pack,2009-07-24,Action,Single-player;Multi-player;Co-op;Downloadable ...,Action,"(6255,)"
1,1257,Killing Floor: Nightfall Character Pack,2010-05-19,Action,Single-player;Multi-player;Co-op;Downloadable ...,Action,"(6255,)"


In [561]:
# export
steam_game.to_csv('Data/steam_game.csv')
dlc_new.to_csv("Data/dlc.csv")
Developer.to_csv('Data/Developer.csv')

## steam_requirements_data

In [54]:
steam_requirement_csv = pd.read_csv('steam_dlc/steam_requirements_data.csv')

In [55]:
steam_csv = pd.read_csv('steam_dlc/steam.csv')

In [56]:
steam_csv

,appid,type,name,required_age,dlc,fullgame,supported_languages,developers,publishers,packages,...,coming_soon,price,review_score,total_positive,total_negative,rating,owners,average_forever,median_forever,tags
0,10,game,Counter-Strike,0,NaN,NaN,"['English', 'French', 'German', 'Italian', 'Ko...",['Valve'],['Valve'],"[574941, 7]",...,False,8.19,9.0,117261.0,3686.0,95.566768,10000000-20000000,10499,202,"['Action', 'FPS', 'Multiplayer', 'Shooter', 'C..."
1,20,game,Team Fortress Classic,0,NaN,NaN,"['English', 'French', 'German', 'Italian', 'Ko...",['Valve'],['Valve'],[29],...,False,3.99,8.0,3896.0,705.0,81.939532,2000000-5000000,1637,23,"['Action', 'FPS', 'Multiplayer', 'Classic', 'H..."
2,30,game,Day of Defeat,0,NaN,NaN,"['English', 'French', 'German', 'Italian', 'Sp...",['Valve'],['Valve'],[30],...,False,3.99,8.0,2794.0,398.0,84.223637,5000000-10000000,169,11,"['FPS', 'World War II', 'Multiplayer', 'Shoote..."
3,40,game,Deathmatch Classic,0,NaN,NaN,"['English', 'French', 'German', 'Italian', 'Ko...",['Valve'],['Valve'],[31],...,False,3.99,6.0,1214.0,308.0,76.485571,5000000-10000000,2632,6,"['Action', 'FPS', 'Classic', 'Multiplayer', 'S..."
4,50,game,Half-Life: Opposing Force,0,NaN,NaN,"['English', 'French', 'German', 'Korean']",['Gearbox Software'],['Valve'],[32],...,False,3.99,9.0,11343.0,519.0,92.916082,5000000-10000000,442,250,"['FPS', 'Action', 'Classic', 'Sci-fi', 'Single..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102499,2028023,dlc,Total War Saga: FALL OF THE SAMURAI – Blood Pack,18,NaN,"{'appid': '201271', 'name': 'A Total War Saga:...","['Czech', 'English', 'French', 'German', 'Ital...",['The Creative Assembly'],['SEGA'],[12437],...,False,NaN,0.0,0.0,0.0,50.000000,0-20000,0,0,NaN
102500,2028055,dlc,Tom Clancy's Ghost Recon Future Soldier - Seas...,0,NaN,"{'appid': '212630', 'name': ""Tom Clancy's Ghos...","['Danish', 'Dutch', 'English', 'French', 'Germ...","['Ubisoft Paris', 'Red Storm Entertainment']",['Ubisoft'],[16540],...,False,19.99,0.0,0.0,0.0,50.000000,0-20000,0,0,NaN
102501,2028056,dlc,Worms Revolution Season Pass,0,NaN,"{'appid': '200170', 'name': 'Worms Revolution'}","['English', 'French', 'German', 'Italian', 'Po...",['Team17 Digital Ltd.'],['Team17 Digital Ltd'],[16652],...,False,14.99,0.0,0.0,0.0,50.000000,0-20000,0,0,NaN
102502,2028062,dlc,Call of Duty®: Black Ops II Season Pass,0,NaN,"{'appid': '202970', 'name': 'Call of Duty®: Bl...","['English', 'French', 'German', 'Italian', 'Sp...",['Treyarch'],['Activision'],[17569],...,False,49.99,0.0,0.0,0.0,50.000000,0-20000,0,0,NaN


In [57]:
steam_csv.isnull().sum()

appid                      0
type                       0
name                       0
required_age               0
dlc                    92808
fullgame               67897
supported_languages      152
developers                41
publishers                40
packages               21351
platforms                  0
categories               106
genres                   193
achievements               0
release_date            6828
supported_audio        52806
coming_soon                0
price                  11834
review_score               0
total_positive             0
total_negative             0
rating                     0
owners                     0
average_forever            0
median_forever             0
tags                   41456
dtype: int64

In [58]:
steam_requirement_csv.shape

(42427, 8)

In [59]:
steam_requirement_csv.head(3)

,appid,pc_requirements,mac_requirements,linux_requirements,pc_minimum,pc_recommended,mac_minimum,mac_recommended
0,10,{'minimum': '\r\n\t\t\t<p><strong>Minimum:</st...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...","500 mhz processor, 96mb ram, 16mb video card, ...",NaN,"OS X Snow Leopard 10.6.3, 1GB RAM, 4GB Hard D...",NaN
1,20,{'minimum': '\r\n\t\t\t<p><strong>Minimum:</st...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...","500 mhz processor, 96mb ram, 16mb video card, ...",NaN,"OS X Snow Leopard 10.6.3, 1GB RAM, 4GB Hard D...",NaN
2,30,{'minimum': '\r\n\t\t\t<p><strong>Minimum:</st...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...","500 mhz processor, 96mb ram, 16mb video card, ...",NaN,"OS X Snow Leopard 10.6.3, 1GB RAM, 4GB Hard D...",NaN


In [60]:
# Inspect null values
steam_requirement_csv.isnull().sum()

appid                     0
pc_requirements           0
mac_requirements          0
linux_requirements        0
pc_minimum               10
pc_recommended        12899
mac_minimum              75
mac_recommended       14275
dtype: int64

According to the information about the dataset, requirement is divided as two parts: minimum and recommended. Thus pc_minimum and pc_recommended are just split from pc_requirements. Therefore, according to our need, we reconstruct requirements by combininng minimum and recommended for PC and MAC. 

In [61]:
pc_minimum = steam_requirement_csv['pc_minimum']
pc_recommended = steam_requirement_csv['pc_recommended']
mac_minimum = steam_requirement_csv['mac_minimum']
mac_recommended = steam_requirement_csv['mac_recommended']

In [62]:
str(pc_minimum.loc[333]) == 'nan'

True

### recombine the pc and mac requirements into strings

In [63]:
pc_requirement = []

for i in range(len(pc_minimum)):
    if str(pc_minimum[i]) == 'nan':
        minimum = 'Minimum Requirement: None'
    else:
        minimum = 'Minimum Requirement: ' + pc_minimum[i]
        
    if str(pc_recommended[i]) == 'nan':
        recommend = 'Recommended Requirement: None'
    else:
        recommend = 'Recommended Requirement: ' + pc_recommended[i]
    
    pc_requirement.append(minimum + '; ' + recommend)    

In [64]:
mac_requirement = []

for i in range(len(mac_minimum)):
    if str(mac_minimum[i]) == 'nan':
        minimum = 'Minimum Requirement: None'
    else:
        minimum = 'Minimum Requirement: ' + mac_minimum[i]
        
    if str(mac_recommended[i]) == 'nan':
        recommend = 'Recommended Requirement: None'
    else:
        recommend = 'Recommended Requirement: ' + mac_recommended[i]
    
    mac_requirement.append(minimum + '; ' + recommend)    

In [65]:
print(len(pc_requirement))

42427


In [66]:
print(len(mac_requirement))

42427


### linux requirement

In [67]:
linux_requirement = steam_requirement_csv['linux_requirements']


In [68]:
len(linux_requirement)

42427

In [69]:
linux_requirement.tail()

42422    {'minimum': '<strong>Minimum:</strong><br><ul ...
42423    {'minimum': '<strong>Minimum:</strong><br><ul ...
42424    {'minimum': '<strong>Minimum:</strong><br><ul ...
42425    {'minimum': '<strong>Minimum:</strong><br><ul ...
42426                                                   []
Name: linux_requirements, dtype: object

In [70]:
linux_requirement[1]

"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual-core from Intel or AMD at 2.8 GHz, 1GB Memory, nVidia GeForce 8600/9600GT, ATI/AMD Radeaon HD2600/3600 (Graphic Drivers: nVidia 310, AMD 12.11), OpenGL 2.1, 4GB Hard Drive Space, OpenAL Compatible Sound Card'}"

In [71]:
linux_requirement[1000]

'{\'minimum\': \'<strong>Minimum:</strong><br><ul class="bb_ul"><li><strong>OS:</strong> Linux Kernel 3.0 or above<br></li><li><strong>Processor:</strong> Core 2 Duo 2GHz or equivalent<br></li><li><strong>Memory:</strong> 4 GB RAM<br></li><li><strong>Graphics:</strong> Intel HD Graphics 4000<br></li><li><strong>Storage:</strong> 12 GB available space<br></li><li><strong>Sound Card:</strong> Yes</li></ul>\', \'recommended\': \'<strong>Recommended:</strong><br><ul class="bb_ul"><li><strong>OS:</strong> Linux Kernel 3.0 or above<br></li><li><strong>Processor:</strong> Quad Core i5 2.5GHz<br></li><li><strong>Memory:</strong> 8 GB RAM<br></li><li><strong>Graphics:</strong> ATI or Nvidia card with 2GB VRAM<br></li><li><strong>Network:</strong> Broadband Internet connection<br></li><li><strong>Storage:</strong> 20 GB available space<br></li><li><strong>Sound Card:</strong> Yes</li></ul>\'}'

In [72]:
# remove the '[]' values in linux requirement
linux_requirement = linux_requirement[linux_requirement != '[]']

In [73]:
linux_requirements = (linux_requirement
                         .str.replace(r'\\[rtn]', '', regex = True)
                         .str.replace(r'<[pbr]{1,2}>', ' ', regex = True)
                         .str.replace(r'<[\/"=\w\s]+>', '', regex = True)
                      )

In [74]:
linux_requirements[1000]

"{'minimum': 'Minimum: OS: Linux Kernel 3.0 or above Processor: Core 2 Duo 2GHz or equivalent Memory: 4 GB RAM Graphics: Intel HD Graphics 4000 Storage: 12 GB available space Sound Card: Yes', 'recommended': 'Recommended: OS: Linux Kernel 3.0 or above Processor: Quad Core i5 2.5GHz Memory: 8 GB RAM Graphics: ATI or Nvidia card with 2GB VRAM Network: Broadband Internet connection Storage: 20 GB available space Sound Card: Yes'}"

In [75]:
# change the string into dictionary through literal_eval
dict_linux_req = [literal_eval(n) for n in linux_requirements]

In [76]:
dict_linux_req[4]

{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual-core from Intel or AMD at 2.8 GHz, 1GB Memory, nVidia GeForce 8600/9600GT, ATI/AMD Radeaon HD2600/3600 (Graphic Drivers: nVidia 310, AMD 12.11), OpenGL 2.1, 4GB Hard Drive Space, OpenAL Compatible Sound Card'}

In [77]:
type(dict_linux_req[4])

dict

In [78]:
# extract the content and put them into strings
cleaned_linux_req = []
for n in dict_linux_req:
    if n:
        if 'minimum' in n.keys():
            minimum = n['minimum']
        else:
            minimum = 'Minimum: None'

        if 'recommended' in n.keys():
            recommend = n['recommended']
        else:
            recommend = 'Recommended: None'
        cleaned_linux_req.append(minimum + '; ' + recommend)
    else:
        cleaned_linux_req.append('None')

In [79]:
len(cleaned_linux_req)

31609

In [80]:
steam_requirement_csv.shape

(42427, 8)

In [81]:
cleaned_steam_requirements = pd.DataFrame({
    'appid': steam_requirement_csv['appid'],
    'pc_requirements': pc_requirement,
    'mac_requirements': mac_requirement,
    'linux_requirements': cleaned_linux_req
})

ValueError: All arrays must be of the same length

In [ ]:
cleaned_steam_requirements.head()

In [ ]:
# export the data
cleaned_steam_requirements.to_csv('Requirement.csv')

## Detail Table

In the ER diagram, there is a table called "Detail" containing detailed information of the games. Here we combined the required detailed information.

Our detailed information table contains appid, languages, short_description, and website url. 

In [602]:
dlc_appid = tuple(steam_csv.query('type == "dlc"')['appid'])

### extract website url for all dlc

In [582]:
dlc_website_data = pd.read_csv('steam_dlc/steam_support_info.csv')

In [611]:
dlc_website_data.isnull().sum()

appid                0
website          41829
support_url      40666
support_email    19883
dtype: int64

In [620]:
dlc_website_data.loc[:3, ['website', 'appid']]

,website,appid
0,NaN,10
1,http://www.dayofdefeat.com/,30
2,NaN,50
3,http://www.half-life.com/,70


In [666]:
website_appid = dlc_website_data['appid'].values

In [667]:
dlc_website_url = []

for appid in dlc_appid:
    if appid in website_appid:
        link = str(dlc_website_data.loc[dlc_website_data['appid'] == appid, 'website'].values[0])
        if link == 'nan':
            link = 'None'
        dlc_website_url.append(link)
    else:
        dlc_website_url.append('None')

In [669]:
len(dlc_website_url)

34632

In [670]:
# extract other columns of dlc detail data
dlc_detail = steam_csv.query('type == "dlc"')[['appid', 'supported_languages']]

In [674]:
dlc_detail['website'] = dlc_website_url

In [676]:
dlc_detail.head(3)

,appid,supported_languages,website
29,1256,['English'],None
30,1257,['English'],http://www.killingfloorthegame.com
162,4856,"['English', 'French', 'German', 'Russian']",http://www.gsc-game.com/


### Add short description data to dlc_detail

In [679]:
dlc_description_data = pd.read_csv('steam_dlc/steam_description_data.csv')

In [690]:
dlc_description_data.head(3)

,appid,detailed_description,about_the_game,short_description
0,10,Play the world's number 1 online action game. ...,Play the world's number 1 online action game. ...,Play the world's number 1 online action game. ...
1,20,One of the most popular online action games of...,One of the most popular online action games of...,One of the most popular online action games of...
2,30,Enlist in an intense brand of Axis vs. Allied ...,Enlist in an intense brand of Axis vs. Allied ...,Enlist in an intense brand of Axis vs. Allied ...


In [693]:
dlc_description_data.loc[dlc_description_data['appid'] == 20, 'short_description'].values[0]

'One of the most popular online action games of all time, Team Fortress Classic features over nine character classes -- from Medic to Spy to Demolition Man -- enlisted in a unique style of online team warfare. Each character class possesses unique weapons, items, and abilities, as teams compete online in a variety of game play modes.'

In [696]:
dlc_short_description = []
description_appid = dlc_description_data['appid'].values

for appid in dlc_appid:
    if appid in description_appid:
        des = dlc_description_data.loc[dlc_description_data['appid'] == appid, 'short_description'].values[0]
        dlc_short_description.append(des)
    else:
        dlc_short_description.append('None')

In [697]:
len(dlc_short_description)

34632

In [699]:
dlc_detail['short_description'] = dlc_short_description

In [700]:
dlc_detail.head(3)

,appid,supported_languages,website,short_description
29,1256,['English'],None,The Outbreak Character Pack adds 4 new and uni...
30,1257,['English'],http://www.killingfloorthegame.com,The Nightfall Character Pack adds 4 more new a...
162,4856,"['English', 'French', 'German', 'Russian']",http://www.gsc-game.com/,This expansion contains four single-player cam...


In [704]:
dlc_detail.isnull().sum()

appid                    0
supported_languages    129
website                  0
short_description        9
dtype: int64

In [706]:
dlc_detail.loc[dlc_detail['short_description'].isnull(), 'short_description'] = 'None'

### clean language
to be a string contain supported languages separated by ;

In [709]:
dlc_detail.loc[dlc_detail['supported_languages'].isnull(), 'supported_languages'].head()

563    NaN
581    NaN
660    NaN
809    NaN
844    NaN
Name: supported_languages, dtype: object

In [710]:
dlc_detail.loc[dlc_detail['supported_languages'].isnull(), 'supported_languages'] = 'Not Available'

In [701]:
print('supported_language datatype:', type(dlc_detail['supported_languages'][29]))

supported_language datatype: <class 'str'>


In [703]:
dlc_detail['supported_languages'].isnull()

29                                              ['English']
30                                              ['English']
162              ['English', 'French', 'German', 'Russian']
239       ['English', 'French', 'German', 'Italian', 'Sp...
240       ['English', 'French', 'German', 'Italian', 'Sp...
                                ...                        
102499    ['Czech', 'English', 'French', 'German', 'Ital...
102500    ['Danish', 'Dutch', 'English', 'French', 'Germ...
102501    ['English', 'French', 'German', 'Italian', 'Po...
102502    ['English', 'French', 'German', 'Italian', 'Sp...
102503    ['English', 'French', 'German', 'Italian', 'Ja...
Name: supported_languages, Length: 34632, dtype: object

In [719]:
dlc_languages = []

for item in dlc_detail['supported_languages']:
    if item != 'Not Available':
        item_lst = literal_eval(item)
        dlc_languages.append(';'.join(item_lst))
    else:
        dlc_languages.append(item)

In [721]:
len(dlc_languages)

34632

In [722]:
dlc_detail['languages'] = dlc_languages

In [724]:
dlc_detail = dlc_detail.drop('supported_languages', axis = 1)

In [725]:
dlc_detail.head(3)

,appid,website,short_description,languages
29,1256,None,The Outbreak Character Pack adds 4 new and uni...,English
30,1257,http://www.killingfloorthegame.com,The Nightfall Character Pack adds 4 more new a...,English
162,4856,http://www.gsc-game.com/,This expansion contains four single-player cam...,English;French;German;Russian


### Now collect detail information for the steam game

In [727]:
steam_detail = steam_basic.loc[:, ('appid', 'languages', 'short_description', 'website')]

In [728]:
steam_detail.head(3)

,appid,languages,short_description,website
0,10,"English, French, German, Italian, Spanish - Sp...",Play the world's number 1 online action game. ...,
1,1000000,"English, Korean, Simplified Chinese",ASCENXION is a 2D shoot 'em up game where you ...,
2,1000010,"Simplified Chinese, English, Japanese, Traditi...","Enter a labyrinth that moves as you move, wher...",


**We clean this dataset so that it has similar format as the dlc dataset in the corresponding columns**

In [729]:
steam_detail.isnull().sum()

appid                0
languages            0
short_description    0
website              0
dtype: int64

**Clean 'languages'**

In [733]:
print('type \'languages\':', type(steam_detail['languages'][0]))

type 'languages': <class 'str'>


In [734]:
steam_detail['languages'][0]

'English, French, German, Italian, Spanish - Spain, Simplified Chinese, Traditional Chinese, Korean'

In [735]:
cleaned_steam_languages = steam_detail['languages'].str.replace(', ', ';')

In [736]:
cleaned_steam_languages[:5]

0    English;French;German;Italian;Spanish - Spain;...
1                    English;Korean;Simplified Chinese
2    Simplified Chinese;English;Japanese;Traditiona...
3                                              English
4                                   Simplified Chinese
Name: languages, dtype: object

**Clean 'website'**

In [737]:
steam_detail['website'][0]

''

In [738]:
# replace all the '' into 'None'
steam_detail.loc[steam_detail['website'] == '', 'website'] = 'None'

In [742]:
steam_detail['website'][:6]

0                                 None
1                                 None
2                                 None
3    http://www.cookservedelicious.com
4                                 None
5                                 None
Name: website, dtype: object

**Clean 'short_description'** 

In [755]:
(steam_detail['short_description'] == '').sum()

37

In [753]:
steam_detail.loc[steam_detail['short_description'] == '', 'short_description'] = 'None'

In [756]:
(steam_detail['short_description'] == '').sum()

0

**Switch the 'languages' column of steam_detail into the cleaned one, and then concat the two detail tables to be ready as the complete detail table for both steam game and dlc.**

In [757]:
steam_detail = steam_detail.drop('languages', axis = 1)

In [758]:
steam_detail['languages'] = cleaned_steam_languages

In [759]:
steam_detail.head(3)

,appid,short_description,website,languages
0,10,Play the world's number 1 online action game. ...,None,English;French;German;Italian;Spanish - Spain;...
1,1000000,ASCENXION is a 2D shoot 'em up game where you ...,None,English;Korean;Simplified Chinese
2,1000010,"Enter a labyrinth that moves as you move, wher...",None,Simplified Chinese;English;Japanese;Traditiona...


In [760]:
dlc_detail.head(3)

,appid,website,short_description,languages
29,1256,None,The Outbreak Character Pack adds 4 new and uni...,English
30,1257,http://www.killingfloorthegame.com,The Nightfall Character Pack adds 4 more new a...,English
162,4856,http://www.gsc-game.com/,This expansion contains four single-player cam...,English;French;German;Russian


In [761]:
steam_detail = steam_detail[['appid', 'languages', 'short_description', 'website']]
dlc_detail = dlc_detail[['appid', 'languages', 'short_description', 'website']]

In [762]:
steam_detail.head(3)

,appid,languages,short_description,website
0,10,English;French;German;Italian;Spanish - Spain;...,Play the world's number 1 online action game. ...,None
1,1000000,English;Korean;Simplified Chinese,ASCENXION is a 2D shoot 'em up game where you ...,None
2,1000010,Simplified Chinese;English;Japanese;Traditiona...,"Enter a labyrinth that moves as you move, wher...",None


In [764]:
dlc_detail.head(3)

,appid,languages,short_description,website
29,1256,English,The Outbreak Character Pack adds 4 new and uni...,None
30,1257,English,The Nightfall Character Pack adds 4 more new a...,http://www.killingfloorthegame.com
162,4856,English;French;German;Russian,This expansion contains four single-player cam...,http://www.gsc-game.com/


In [766]:
detail = pd.concat([steam_detail, dlc_detail], join = 'inner', ignore_index = True)

In [808]:
detail.head()

,appid,languages,short_description,website
0,10,English;French;German;Italian;Spanish - Spain;...,Play the world's number 1 online action game. ...,None
1,1000000,English;Korean;Simplified Chinese,ASCENXION is a 2D shoot 'em up game where you ...,None
2,1000010,Simplified Chinese;English;Japanese;Traditiona...,"Enter a labyrinth that moves as you move, wher...",None
3,1000030,English,"Cook, serve and manage your food truck as you ...",http://www.cookservedelicious.com
4,1000040,Simplified Chinese,这是一款打击感十足的细胞主题游戏！操作简单但活下去却不简单，“你”作为侵入人体的细菌病毒，通...,None


In [768]:
detail.to_csv('Data/Detail.csv')

## Build connection from dlc to steam_game

Inside the dlc_table, we decided to add the parent game id for each dlc referring from the dlc table to the steam_game table.

In [769]:
dlc_new.head(3)

,appid,name,release_date,genre,categories,tags,developer_id
0,1256,Killing Floor Outbreak Character Pack,2009-07-24,Action,Single-player;Multi-player;Co-op;Downloadable ...,Action,"(6255,)"
1,1257,Killing Floor: Nightfall Character Pack,2010-05-19,Action,Single-player;Multi-player;Co-op;Downloadable ...,Action,"(6255,)"
2,4856,Cossacks: Campaign Expansion,2011-01-21,Strategy,Single-player;Downloadable Content,Strategy,"(4984,)"


We want to add a column called 'parent_appid' in this dataset. In the steam_csv, the column 'fullgame' contains information of the appid and name of its parent game. Therefore, we extract it. 

In [778]:
parent_info = steam_csv.query('type == "dlc"').loc[:, ('appid', 'fullgame')]

In [779]:
parent_info.head(3)

,appid,fullgame
29,1256,"{'appid': '1250', 'name': 'Killing Floor'}"
30,1257,"{'appid': '1250', 'name': 'Killing Floor'}"
162,4856,"{'appid': '4850', 'name': 'Cossacks: Back to W..."


In [780]:
parent_info.isnull().sum()

appid        0
fullgame    26
dtype: int64

In [782]:
print('type of column \'fullgame\'', type(parent_info['fullgame'][29]))

type of column 'fullgame' <class 'str'>


In [786]:
parent_info.loc[parent_info['fullgame'].isnull(), :].head(3)

,appid,fullgame
1238,50142,NaN
1929,205093,NaN
3045,230889,NaN


In [787]:
parent_info.loc[parent_info['fullgame'].isnull(), 'fullgame'] = 'Not Available'

In [797]:
parent_info.isnull().sum()

appid       0
fullgame    0
dtype: int64

In [812]:
parent_info = parent_info.reset_index()

In [822]:
parent_info.iloc[178:180, :]

,index,appid,fullgame
178,1060,42722,"{'appid': '42700', 'name': 'Call of Duty®: Bla..."
179,1061,42800,25930.0


In [818]:
int(float('1250.0'))

1250

In [832]:
parent_id = []
for item in parent_info['fullgame']:
    if item != 'Not Available':
        try:
            parent_id.append(int(literal_eval(item)['appid']))
        except:
            parent_id.append(int(float(item)))
    else:
        parent_id.append(np.nan)

In [836]:
dlc_new['parent_id'] = parent_id

In [837]:
dlc_new.head(3)

,appid,name,release_date,genre,categories,tags,developer_id,parent_id
0,1256,Killing Floor Outbreak Character Pack,2009-07-24,Action,Single-player;Multi-player;Co-op;Downloadable ...,Action,"(6255,)",1250.0
1,1257,Killing Floor: Nightfall Character Pack,2010-05-19,Action,Single-player;Multi-player;Co-op;Downloadable ...,Action,"(6255,)",1250.0
2,4856,Cossacks: Campaign Expansion,2011-01-21,Strategy,Single-player;Downloadable Content,Strategy,"(4984,)",4850.0


In [843]:
dlc_new.to_csv('Data/dlc.csv')